# Weatherdata for IPM Plateform

## 1. import weather data modules

In [1]:
from weatherdata.ipm import WeatherDataHub 
import pandas
pandas.set_option('display.max_rows',10)

## 2. Access to weatherdatasource available on the platefom

In [2]:
ws=WeatherDataHub()

In [3]:
ws.list_resources

,name,description,parameters
0,Met Norway Locationforecast,9-day forecasts for the entire planet. 2.5 km ...,"{'common': [1001, 3001, 2001, 4002], 'optional..."
1,Met Éireann Locationforecast,9-day forecasts for Ireland.,"{'common': [1001, 3001, 2001, 4002], 'optional..."
2,DMI Pointweather service,Seasonal weather data and forecasts for Denmar...,"{'common': [1002, 1112, 2001, 3002, 3101, 4002..."
3,SLU Lantmet service,Seasonal weather data and forecasts for Sweden...,"{'common': [1002, 1003, 1004, 2001, 3002, 3003..."
4,Deutsche Wetterdienst location forecast by IPM...,27 hour weather forecasts for Germany and surr...,"{'common': [1001, 3001, 2001, 4002], 'optional..."
...,...,...,...
10,Landbruksmeteorologisk tjeneste,Weather station network covering major agricul...,"{'common': [1002, 1003, 1004, 3002, 2001, 4003..."
11,MeteoBot API,Network of privately owned weather stations. A...,"{'common': [1001, 3001, 2001, 4002], 'optional..."
12,Fruitweb,Network of privately owned weather stations of...,"{'common': [1001, 3001, 2001, 4002], 'optional..."
13,Metos,Network of privately owned weather stations of...,"{'common': [1001, 3001, 2001, 4002], 'optional..."


## 3. Get ressource for a specific weatherdataresource 

To connect to the meteo resource, simply enter the name of the meteo resource in the get_ressource function. 

In [4]:
fmi=ws.get_ressource(name='Finnish Meteorological Institute measured data')

TypeError: get_ressource() missing 1 required positional argument: 'df'

Once connected to the resource various functions are available. You can check 
* the available stations for the resource with station_ids function. This function This function returns a dataframe of the stations available for the resource with their names, their identifiers and the coordinates of the station (latitude, longitude)
* the weather parameters accepted by the resource  with parameters function. This function return common and optional weather parameter 
* Get Weatherdata in xarray dataset with attribute or json format (more description of this function below)

In [ ]:
fmi.stations


In [ ]:
fmi.parameter

### Get weather data

According to weather resources differents parameters can be used:
* *for historic weatherdata*
    * parameters: list of weather parameters available for the resource selected
    * station_id: list of station id available for the resource selected
    * timeStart: Start date of the request
    * timeEnd: End date of the request
    * timezone: The timezone
    * format: 'ds' to obtain a xarray dataset or 'json' to obtain the json     
    
    
* *for forecasts weather resources*
    * latitude: list of latitude
    * longitude: list of longitude
    * altitude: list of altitude

#### Example for historic weather ressources 
* for one station_id

In [ ]:
# for one station_ids
ds=fmi.data(parameters=[1002,3002],
            stationId=[101533],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            display='ds',
            varname='id')
ds

In [ ]:
ds.to_dataframe()

* for several station_ids

In [ ]:
ds=fmi.data(parameters=[1002,3002],
            stationId=[101533,101185],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            display='ds',varname='name')
ds

In [ ]:
ds.to_dataframe()

#### For forecast weather resources
* example for one latitude, longitude, altitude

In [ ]:
norway=ws.get_ressource(name='Met Norway Locationforecast')

In [ ]:
ds=norway.data(latitude=[67.2828], longitude=[14.3711], altitude=[70],format='ds',varname='id')
ds

In [ ]:
ds=norway.data(latitude=[67.2828,61.27], longitude=[14.3711,25.52], altitude=[70, 0],format='ds',varname="name")
ds

In [ ]:
ds.to_dataframe()

**From ds you can see data as dataframe with function to_dataframe and exclude na value with dropna**
for more information on the http://xarray.pydata.org/en/stable/index.html

#### Example of weatherdatasource with credentials

In [ ]:
fruitdevis=ws.get_ressource(name='Fruitweb')
fruitdevis.data(parameters=[1002,3002],stationId=[536], timeStart='2021-02-01',timeEnd='2021-03-01',credentials={"userName":"","password":"GF90esoleo"},varname="id")

# Metpy test

In [ ]:
import metpy.calc as mpcalc
from metpy.units import units
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs

In [ ]:
fmi=ws.get_ressource(name='Finnish Meteorological Institute measured data')
list_station=list(fmi.station_ids().id.astype("int"))


In [ ]:
ds=fmi.data(parameters=[1002,3002],
            stationId=list_station[:25],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            format='ds',varname='id',savecache=True,usecache=True)

ds=ds.astype("float")
# ds['1002'].attrs['description']=str(ds['1002'].attrs['description'])
# ds['3002'].attrs['description']=str(ds['1002'].attrs['description'])
# ds.to_netcdf("toto.nc")
ds

## Metpy station plot

### Plot stations on map

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

from metpy.plots import StationPlot

In [ ]:
data=ds.isel(time=0)
data=data.to_dataframe()
range(len(data.index.levels[0].values))

In [ ]:
proj = ccrs.LambertConformal()
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=proj)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAKES)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS)


stationplot = StationPlot(ax, data.index.levels[1].values, data.index.levels[2].values,
                          clip_on=True, transform=ccrs.Geodetic(), fontsize=12)
stationplot.plot_parameter(location="NW",parameter=data['1002'].values,color='red')
stationplot.plot_parameter(location="SW",parameter=data['3002'].values,color='blue')
stationplot.plot_text((5, 0),data.index.levels[0].values)

### Plot temperature on map

#### example metpy XArray Projection Handling

In [ ]:
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr

# Any import of metpy will activate the accessors
from metpy.cbook import get_test_data

ds1 = xr.open_dataset(get_test_data('narr_example.nc', as_file_obj=False))
data_var = ds1.metpy.parse_cf('Temperature')
ds1

In [ ]:
x = data_var.x
y = data_var.y
im_data = data_var.isel(time=0).sel(isobaric=1000.)

fig = plt.figure(figsize=(14, 14))
ax = fig.add_subplot(1, 1, 1, projection=data_var.metpy.cartopy_crs)

ax.imshow(im_data, extent=(x.min(), x.max(), y.min(), y.max()),
          cmap='RdBu', origin='lower' if y[0] < y[-1] else 'upper')
ax.coastlines(color='tab:green', resolution='10m')
ax.add_feature(cfeature.LAKES.with_scale('10m'), facecolor='none', edgecolor='tab:blue')
ax.add_feature(cfeature.RIVERS.with_scale('10m'), edgecolor='tab:blue')

plt.show()

#### test with IPM data

In [ ]:
import numpy

data_var = ds.metpy.parse_cf('1002')
y= ds.lat
x= ds.lon

data_var , x , y
#xr.merge([data_var,x])

# im_data = data_var.isel(time=0)

# fig = plt.figure(figsize=(14, 14))
# ax = fig.add_subplot(1, 1, 1, projection=data_var.metpy.cartopy_crs)

# ax.imshow(numpy.float64(im_data),
#            cmap='RdBu')
# ax.coastlines(color='tab:green', resolution='10m')
# ax.add_feature(cfeature.LAKES.with_scale('10m'), facecolor='none', edgecolor='tab:blue')
# ax.add_feature(cfeature.RIVERS.with_scale('10m'), edgecolor='tab:blue')
# ax.add_feature(cfeature.LAND)
# ax.add_feature(cfeature.OCEAN)
# ax.add_feature(cfeature.STATES)
# ax.add_feature(cfeature.BORDERS)
#data_var


In [ ]:
da = data_var.to_dataset().assign_coords({"lon":x,"lat":y})

im_data = da.isel(time=0)
fig = plt.figure(figsize=(14, 14))
ax = fig.add_subplot(1, 1, 1, projection=da.metpy.cartopy_crs)

ax.imshow(numpy.float64(im_data),
           cmap='RdBu')
ax.coastlines(color='tab:green', resolution='10m')
ax.add_feature(cfeature.LAKES.with_scale('10m'), facecolor='none', edgecolor='tab:blue')
ax.add_feature(cfeature.RIVERS.with_scale('10m'), edgecolor='tab:blue')
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS)

In [ ]:
from metpy.interpolate import interpolate_to_grid, remove_nan_observations, remove_repeat_coordinates
import numpy as np

to_proj = ccrs.LambertConformal()

lon = ds.lon.values
lat = ds.lat.values

xp, yp, _ = to_proj.transform_points(ccrs.Geodetic(), lon, lat).T

x_masked, y_masked, temp = remove_nan_observations(xp, yp, np.float64(ds['1002'].isel(time=0).values))

In [ ]:
tempgridx, tempgridy, temp = interpolate_to_grid(x_masked, y_masked, temp,
                                              interp_type='cressman', minimum_neighbors=1,
                                              search_radius=400000, hres=100000)

temp


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
levels = list(range(-20, 20, 1))
cmap = plt.get_cmap('inferno')
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

fig = plt.figure(figsize=(20, 10))
view = fig.add_subplot(1, 1, 1, projection=to_proj)

#view.set_extent([lon.min(), lat.min(), lon.max(), lat.min()])
view.add_feature(cfeature.STATES.with_scale('50m'))
view.add_feature(cfeature.OCEAN)
view.add_feature(cfeature.COASTLINE.with_scale('50m'))
view.add_feature(cfeature.BORDERS, linestyle=':')

#cs = view.contour(tempgridx, tempgridy, temp, colors='k', levels=list(range(990, 1034, 4)))
#view.clabel(cs, inline=1, fontsize=12, fmt='%i')

mmb = view.pcolormesh(tempgridx, tempgridy, temp, cmap=cmap, norm=norm)
fig.colorbar(mmb, shrink=.4, pad=0.02, boundaries=levels)

#view.barbs(windgridx, windgridy, uwind, vwind, alpha=.4, length=5)

view.set_title('surface temperature map')

plt.show()

In [ ]:
ds.attrs['timeStart']=str(ds.attrs['timeStart'])
ds.attrs['timeEnd']=str(ds.attrs['timeEnd'])
ds['1002'].attrs['description']=str(ds['1002'].attrs['description'])
ds['3002'].attrs['description']=str(ds['1002'].attrs['description'])
ds.to_netcdf("toto.nc")


### Test with xarray tutorial

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
from matplotlib import pyplot as plt

In [ ]:
dsxarray = xr.tutorial.open_dataset("rasm").load()
dsxarray

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(14, 4))
dsxarray.xc.plot(ax=ax1)
dsxarray.yc.plot(ax=ax2)

In [ ]:
dsxarray.Tair[0].plot()

In [ ]:
dsxarray2 = xr.tutorial.load_dataset("air_temperature")

air = dsxarray2.air.isel(time=[0, 724]) - 273.15
air
map_proj = ccrs.LambertConformal(central_longitude=-95, central_latitude=45)
p = air.plot(
    transform=ccrs.PlateCarree(),  # the data's projection
    col="time",
    col_wrap=1,  # multiplot settings
    aspect=dsxarray2.dims["lon"] / dsxarray2.dims["lat"],  # for a sensible figsize
    subplot_kws={"projection": map_proj},
)  # the plot's projection

for ax in p.axes.flat:
    ax.coastlines()
    ax.set_extent([-160, -30, 5, 75])
dsxarray2

#### test with IPM data

In [ ]:
ds=fmi.data(parameters=[1002,3002],
            stationId=list_station[:25],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            format='ds',varname='id',savecache=True,usecache=True) 

In [ ]:
fig, ax1 = plt.subplots(ncols=1, figsize=(4, 4))
ax1.plot(ds.lon, ds.lat)

In [ ]:
ds['1002'][0].plot()
fig, ax1 = plt.subplots(ncols=1, figsize=(4, 4))
ax1.plot(ds.lon, ds.lat)

import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xrb

In [ ]:
ds=fmi.data(parameters=[1002,3002],
            stationId=[101533],
            timeStart='2020-06-12',
            timeEnd='2020-07-03',
            timeZone='UTC',
            format='ds',
            varname='id')


In [ ]:
temp = ds['1002']*units.degC
hum=ds['3002']*units.percent

In [ ]:
mpcalc.dewpoint_from_relative_humidity(temp, hum)